In [24]:
import torch
from torchvision import models
from torchvision import transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import os

plt.ion()

time: 3.8 ms (started: 2023-08-01 07:11:33 +00:00)


In [23]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00
time: 269 µs (started: 2023-08-01 07:08:15 +00:00)


In [ ]:
# Define the batch size and number of epochs
batch_size = 32
num_epochs = 50

In [ ]:
dir(models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

In [ ]:
# Define the transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Download the CIFAR10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 55887434.14it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=32,
                                         shuffle=True)

In [ ]:
# Define the model
model = models.resnet18(weights=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 210MB/s]


In [ ]:
for param in model.parameters():#Freeze the model's parameters to prevent them from being updated during training
    param.requires_grad = False

In [ ]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
!nvidia-smi

Tue Aug  1 05:15:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Get GPU memory usage
gpu_memory_used = torch.cuda.memory_allocated(device=device)
print(f"GPU memory used: {gpu_memory_used / 1024**3:.2f} GB")

GPU memory used: 0.00 GB


In [ ]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
# Train the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainloader)}")

Epoch 1/50, Loss: 0.8641267588179766
Epoch 2/50, Loss: 0.657019881077554
Epoch 3/50, Loss: 0.6291304499254117
Epoch 4/50, Loss: 0.6094081316784415
Epoch 5/50, Loss: 0.5992941077100262
Epoch 6/50, Loss: 0.5924865795741536
Epoch 7/50, Loss: 0.5857563023661011
Epoch 8/50, Loss: 0.5794096216321067
Epoch 9/50, Loss: 0.5759952437473427
Epoch 10/50, Loss: 0.5758807463136455
Epoch 11/50, Loss: 0.5721127243279953
Epoch 12/50, Loss: 0.5697652187081613
Epoch 13/50, Loss: 0.5686685838191385
Epoch 14/50, Loss: 0.5636609235560368
Epoch 15/50, Loss: 0.5680984018212012
Epoch 16/50, Loss: 0.5611909193082719
Epoch 17/50, Loss: 0.563598512220825
Epoch 18/50, Loss: 0.5631057004934691
Epoch 19/50, Loss: 0.5591615550386853
Epoch 20/50, Loss: 0.5592999503884037
Epoch 21/50, Loss: 0.5582513847624881
Epoch 22/50, Loss: 0.5586576139457853
Epoch 23/50, Loss: 0.5558171214122308
Epoch 24/50, Loss: 0.5582838509319993
Epoch 25/50, Loss: 0.5544578715615447
Epoch 26/50, Loss: 0.5542877086903601
Epoch 27/50, Loss: 0.55

In [19]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

Accuracy on test set: 81.45%


In [21]:
#Get GPU memory usage
gpu_memory_used = torch.cuda.memory_allocated(device=device)
print(f"GPU memory used: {gpu_memory_used / 1024**3:.2f} GB")

GPU memory used: 0.07 GB


In [28]:
model.eval()

# Load and preprocess the new image
new_image_path = "/content/cat-portrait.jpg"
img = Image.open(new_image_path)
input_tensor = transform(img).unsqueeze(0)  # Add a batch dimension to the tensor
input_tensor = input_tensor.to(device)

# Perform inference
with torch.no_grad():
    output = model(input_tensor)

# Interpret the output to get the predicted class label
_, predicted_class = torch.max(output, 1)

# Get the class label from the predicted index
class_labels = [
    "airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"
]
predicted_label = class_labels[predicted_class.item()]

print(f"The model predicts the image as: {predicted_label}")

The model predicts the image as: cat
time: 23.9 ms (started: 2023-08-01 07:29:11 +00:00)
